<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

Cloning into 'tol403-lokaverkefni'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 45 (delta 14), reused 11 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 4.05 MiB | 6.45 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [11]:
import pandas as pd
import folium
from folium import Map, Marker, PolyLine, features, RegularPolygonMarker, DivIcon
from folium.plugins import PolyLineTextPath
import math
import networkx as nx
import time
import concurrent.futures

In [3]:
# 2.3.1

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep = "\t")
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep = "\t")

charging_station_nodes = {323346405, 87120378, 2374444198, 1345740157, 2351742223}

coords = {
    row['osmid']: (row['y'], row['x'])  # Folium notar (lat, lon)
    for _, row in nodes.iterrows()
}

# búum til graf með stefnu og þyngd fyrir edges
G = nx.DiGraph()
for _, row in edges.iterrows():
    G.add_edge(row['u'], row['v'], weight=row['length'])

In [4]:
# 2.3.2
def shortest_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.dijkstra_path_length(graph, source=node_id, target=charger_id, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station

In [5]:
# 2.3.3

def create_map():
    # Búum til base map með miðju í average location.
    center_lat = nodes['y'].mean()
    center_lon = nodes['x'].mean()
    m = folium.Map(location = [center_lat, center_lon], zoom_start= 12)

    # reiknum gráður fyrir örina sjálfa
    def calculate_angle(start, end):
        dy = end[0] - start[0]
        dx = end[1] - start[1]
        angle = math.degrees(math.atan2(dy, dx))
        return angle

    start_time = time.time()

    # setjum nodes á kortið - blátt=nonprimary rautt=primary og appelsínugult= hleðslustöð.
    for _, row in nodes.iterrows():
        node_id = row['osmid']

        # Decide color and popup content
        if node_id in charging_station_nodes:
            color = "orange"
            popup_text = f"🔌 Hleðslustöð <br>Node ID: {node_id}"
        else:
            color = "red" if row['primary'] else "blue"
            distance, closest = shortest_distance_to_charger(node_id, G, charging_station_nodes)
            popup_text = f"🚗 Node ID: {node_id}<br>Primary: {row['primary']}<br>" \
                        f"Fjarlægð frá næstu hleðslustöð: {distance:.2f} meters<br>Næsta hleðslustöð: {closest}"

        folium.CircleMarker(
            location=[row['y'], row['x']],
            radius=4,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=folium.Popup(popup_text, max_width=250)
        ).add_to(m)

    print(f"Keyrsla tók {int(time.time() - start_time)}s")

    # setjum edges með stefnu á kortið
    for _, row in edges.iterrows():
        if row['u'] in coords and row['v'] in coords:
            start = coords[row['u']]
            end = coords[row['v']]

            # línan
            PolyLine(
                locations=[start, end],
                color='gray',
                weight=1,
                opacity=0.5,
                popup=row.get('name', '')
            ).add_to(m)

            # Reiknum gráðu á ör-iconinu og setjum svo á endann á línunni
            # vantar að laga betur, gerir kortið mjög hægt líka.
            """angle = calculate_angle(start, end)
            folium.map.Marker(
                location=end,
                icon=DivIcon(
                    icon_size=(150, 36),
                    icon_anchor=(7, 20),
                    html=f'<div style="transform: rotate({angle}deg); color: gray; font-size: 16px;">&#8594;</div>'
                )
            ).add_to(m)"""


    output_file = "kort.html"
    m.save(output_file)
    print(f"Kort geymt i skra: {output_file}")

start_time = 0
run_time = 0

create_map()

Keyrsla tók 487s
Kort geymt i skra: kort.html


In [6]:
# 2.3.4

## it's in the code above

In [7]:
# 2.3.5

## Heuristic for A*
## Needlessly complicated on such a small scale
## Calculate the distance between two points on the surface of a sphere
## It's somewhat off up here since the Earth isn't a perfect sphere

def calc_spherical_distance(node_id, target_id):
    lat_source,lon_source = coords[node_id]
    lat_target,lon_target = coords[target_id]

    radius = 6373.0

    delta_lat = math.radians(lat_target) - math.radians(lat_source)
    delta_lon = math.radians(lon_target) - math.radians(lon_source)

    a = math.sin(delta_lat / 2)**2 + math.cos(lat_source) * math.cos(lat_target) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000 #metres

    return distance

def astar_distance_to_charger(node_id, graph, charger_nodes):
    min_distance = float('inf')
    closest_station = None
    for charger_id in charger_nodes:
        try:
            dist = nx.astar_path_length(graph, source=node_id, target=charger_id, heuristic=calc_spherical_distance, weight='weight')
            if dist < min_distance:
                min_distance = dist
                closest_station = charger_id
        except (nx.NetworkXNoPath, nx.NodeNotFound):
            continue
    return min_distance, closest_station


## Return the sum for later use
def run_astar(charging_station_nodes, benchmark):
    distance_sum = 0.0
    if benchmark == True:
        start_time = time.time()

    for _, row in nodes.iterrows():
        node_id = row['osmid']
        distance, closest = astar_distance_to_charger(node_id, G, charging_station_nodes)

        ## Ignore unreachable paths
        if distance != float("inf"):
            distance_sum += distance

    if benchmark == True:
        print(f"Keyrsla tók {int(time.time() - start_time)}s")

    return distance_sum

run_astar(charging_station_nodes, True)

Keyrsla tók 331s


52496231.65324983

In [ ]:
# 2.3.6

## For all primary nodes, run A* and check whichever is the closest to all nodes overall

def process_node(node):
    distance = run_astar([node.osmid], False)
    print(f"Distance sum from {node.osmid} was {distance}")
    return node, distance

def place_optimal_charger():

    # first get primary nodes
    primary_nodes = []
    for _, row in nodes.iterrows():
        if row['primary'] == True:
            primary_nodes.append(row)

    start_time = time.time()

    # Multi-processing so it doesn't take a decade
    # Colab only has 2 cores so it's not THAT much faster
    results = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for node, distance in executor.map(process_node, primary_nodes):
            results.append((node, distance))

    best_node, best_distance = min(results, key=lambda x: x[1])

    print(f"Keyrsla tók {int(time.time() - start_time)}s")
    return best_node, best_distance

place_optimal_charger()

Distance sum from 12885924 was 103926680.1080052
Distance sum from 12885876 was 98018295.05329408
Distance sum from 12885930 was 98241165.93938297
Distance sum from 12885952 was 97933738.12789126
Distance sum from 12885979 was 97906567.29132639
Distance sum from 12885991 was 98154817.19460295
Distance sum from 12886003 was 103851397.87259687
Distance sum from 12886027 was 98009926.37293293
Distance sum from 14581636 was 122869154.79011011
Distance sum from 14581771 was 122477809.08395684
Distance sum from 14581824 was 123375167.8341863
Distance sum from 14581900 was 123146914.82818255
Distance sum from 14581950 was 123082819.95090888
Distance sum from 14581956 was 116186341.8013952
